# Тема 2. Генерация простых чисел
## Лабораторная работа №2

Гершов Михаил Дмитриевич

$N$ - ограничение на длину входных данных

### Свидетели простоты

Пусть ${n\in\mathbb{N}}$ нечетное число и 
$$
  n-1=2^st,
$$
$t$ — нечетное. 

Число ${a\in\mathbb{N}},$ ${0<a<n,}$ называется свидетелем простоты числа $n,$ если выполняются оба условия:

$1)$ $НОД(a,n)=1;$

$2)$ Справедливо хотя бы одно из сравнений 
$$a^t\equiv1\mod n,\qquad a^{t}\equiv-1\mod
    n,\qquad a^{2t}\equiv-1\mod
    n,\qquad\dots,\qquad a^{2^{s-1}t}\equiv-1\mod
    n.
$$

In [142]:
import math, random, sympy


def TrialDivision(n):
    d = 2
    while d <= n**0.5:
        if n % d == 0:
            return False
        d += 1
    return True

def st(n):
    s = 0
    t = n - 1
    while t % 2 == 0:
        s += 1
        t = t // 2
    return s, t

# тест на свидетельство простоты
def WitnessQ(a, n, s = 0, t = 0):  
    if n % 2 == 0:
        return False
    if s == 0:
        s, t = st(n)    
    if math.gcd(a,n) > 1:
        return False
    b = pow(a,t,n)
    if b == 1 or b == n - 1:  # модуль - число положительное
        return True
    for _ in range(1,s):
        b = b**2 % n
        if b == n - 1:
            return True
    return False

def ExtendedGCD(a, b):  # расширенный алгоритм Евклида
    m11, m12 = 1, 0
    m21, m22 = 0, 1
    while b:
        q = a // b
        r = a % b
        a, b = b, r
        m11, m12 = m12, m11 - m12*q
        m21, m22 = m22, m21 - m22*q 
    return (m11,m21)

def looking_inverse_element(a, m):
    if math.gcd(a, m) == 1:
        return ExtendedGCD(a, m)[0] % m 
    else:
        return 'Обратного элемента не существует'
    
def fast_power_leftward(a, d, m):
    
    """ the "left to right" algorithm """
    
    b = a

    for i in bin(d)[3:]:  # 0-я цифра не учитывается
        b = b ** 2 * a ** int(i) % m
        
    return b

# Задание 1.

Являются ли числа 7957, 8321, 13747, 18721, 19951 псевдопростыми по основанию 2?

Нечётное составное число $n$ называется псевдопростым по основанию $a,$ $1<a<n,$ если  
 $$
 a^{n-1}\equiv1\mod n.
 $$

In [87]:
a = 2;
n = 7957;
pow(2,n - 1,n)

1

In [88]:
n = 8321;
pow(2,n - 1,n)

1

In [89]:
n = 13747;
pow(2,n - 1,n)

1

In [90]:
n = 18721;
pow(2,n - 1,n)

1

In [91]:
n = 19951;
pow(2,n - 1,n)

1

## Задание 2.

Найти число Кармайкла, порядковый номер которого соответствует Вашему порядковому номеру в списке подгруппы.

### Числа Кармайкла

 Составное число $n\in\mathbb{N}$ называется **числом Кармайкла**,     если для всех натуральных $a$  таких, что $1<a<n,$ $НОД(a,n)=1,$ справедливо     сравнение 
$$
  a^{n-1}\equiv1\mod n.
$$

In [92]:
def is_carmichael_number(n):
    if TrialDivision(n):
        return False
    else:
        flag = 0
        for a in range(2,n):
            if math.gcd(a,n) != 1:  # если числа не взаимно простые, то они не проверяются
                continue
            else:
                if pow(a,n - 1,n) != 1:
                    return False
                else:
                    flag += 1
                      
        if flag == 0:
            return False
        else:
            return True

In [140]:
index = 1;
numbers = dict()
n = 5
for number in range(2,10000):
    if is_carmichael_number(number):
        numbers[index] = number
        index += 1
        
print('2-е число Кармайкла равно:', numbers[2])

2-е число Кармайкла равно: 1105


Вычеты по модулю заданного простого числа образуют поле.

Формально, поле — алгебра над множеством $F$, образующая коммутативную группу по сложению $+$ над $F$ с нейтральным элементом 0 и коммутативную группу по умножению над ненулевыми элементами $F\setminus0$, при выполняющемся свойстве дистрибутивности умножения относительно сложения.

# Задание 3.

Оценить количество арифметических операций в Тесте Миллера-Рабина. Оценить временную сложность алгоритма. Является ли он полиномиальным?

In [94]:
def MillerRabinTest(n, k = 10):
    from random import randint
    if n % 2 == 0:
        return False
    s,t = st(n)
    for _ in range(k):
        a = randint(2,n - 1)
        if not WitnessQ(a,n,s,t):
            return False
    return True

Если входное число является чётным, то алгоритм выполнит всего $1$ арифметическую операцию (деление с остатком на 2), если же входное число не является чётным, тогда:

1. В функции $st(n)$ может быть от $5$ до $3*s + 2$ операций.
2. В функции $randint(2,n - 1)$ от $n$ отнимается $1$ — $1$ операция.
3. В функции $WitnessQ(a, n, s, t)$ может быть от $1$ до $(1 + (3*s + 2) + m + 3*s)$ операций.

В сумме $1 + (3*s + 2) + 1 + k*(1 + (3*s + 2) + m + 3*s) = k*(6*s + 3 + m) + 3*s + 4$ операций, где $m$ — порядковый номер остатка в алогоритме Евклида, $s$ — определённое в функции число, $k$ — количество итераций. Таким образом, $$f(n,k) = k*(6*s + 3 + m) + 3*s + 4.$$

Так как $n - 1 = 2^s * t$, при этом наибольшее значение достигается при $t = 1$, то $$s = [\log_{2}{(n - 1)}].$$

Количество итераций в алгоритме Евклида может быть оценено следующим образом: $$m<\frac{\left<n\right>}{\log_2G} + 1.$$

Длина числа вычисляется по формуле: $$\left<z\right>:=\lceil\log_2(|z|+1)\rceil.$$

С учётом этого, получим: $$f(n,k) \leq k*(6*[\log_{2}{(n - 1)}] + 3 + \frac{\left<n\right>}{\log_2G} + 1) + 3*[\log_{2}{(n - 1)}] + 4 =$$

$$k*(2*3*[\log_{2}{(n - 1)}] + 4 + \frac{\left<n\right>}{\log_2G}) + 3*[\log_{2}{(n - 1)}] + 4 =$$

$$[A = 3*[\log_{2}{(n - 1)}] + 4] =$$

$$k*(2*A + \frac{\left<n\right>}{\log_2G}) + A.$$

Для временной сложности справедлива оценка:
$$
  T(N):=\max\limits_{\left<n,k\right>\leq N} f(n,k) \leq \max\limits_{\left<n,k\right>\leq N} (k*(2*A + \frac{\left<n\right>}{\log_2G}) + A) \leq k*f{(N)},
$$
где $f{(N)}$ — некоторая функция от $N$.

Кроме того длина чисел, участвующих в вычислениях алгоритма, ограничена полиномом.

Резюмирая всё вышесказанное, делаем вывод, что **тест Миллера-Рабина является полиномиальным алгоритмом**.

# Задание 4.

Используя Тесте Миллера-Рабина, написать функцию, которая генерирует случайное простое число, состоящее из $b$ бит. Проверить полученное число при помощи функции $isprime(p)$ модуля $sympy$.


In [99]:
def generates_prime_number(b):
    while True:
        n = random.randint(2**(b - 1),2**b - 1)
        if MillerRabinTest(n):
            return n
        else:
            continue

In [103]:
number = generates_prime_number(8)
sympy.isprime(number)

True

# Задание 5.

Пусть $a = 9345883071009581737,  m = 341531$.  

Доказать, что если $n<m$ и число $a$ является свидетелем простоты числа $n,$ то $n$ — простое.
Данный факт используется в коде функции $isprime$. Доказать это утверждение можно при помощи вычислений на компьютере.

In [155]:
a = 9345883071009581737
m = 341531

def n_is_prime_number(a,m):
    while True:
        n = random.randint(2,m - 1)  # гарантия того, что n < m
        if WitnessQ(a,n):  # число a — свидетель простоты числа n
            return n
        else:
            continue

In [157]:
MillerRabinTest(n_is_prime_number(a,m))

True

# Задание 6.

**Теорема Н. Диемитко.**  
Пусть 
$$\;n=2rq+1,\;$$
где $q$ — нечетное простое число, 
$$r\leq 2q+1.$$ 

Если существует такое число $a\in\mathbb{N},$ что
  $$
    a^{n-1}\equiv1\mod n,\qquad a^{2r}\not\equiv 1\mod n,
  $$
  то $n$ — простое число.  

На основе теоремы Диемитко написать алгоритм, который, начиная с малого простого числа, строит случайное простое число, большее $n\in\mathbb{N}.$ Является ли полученный алгоритм: 
- детерминированным алгоритмом?
- вероятностным алгоритмом?
- полиномиальным алгоритмом?

In [123]:
def algorithm_diemitko(b):
    q = generates_prime_number(b)
    r = 1
    while True:
        n = 2*r*q + 1
        for a in range(2,n):
            if pow(a, n - 1, n) == 1 and pow(a, 2*r, n) != 1:
                return generates_prime_number(len(bin(n)[2:]) + 1)  # если n простое, находим случайное простое на 1 бит большее
        r += 1

In [141]:
sympy.isprime(algorithm_diemitko(10))

True

Алгоритм не является детерминированным, поскольку в его коде присутствует генератор случайных чисел (generates_prime_number()). Следовательно, алгоритм является вероятностным.

Так как длина чисел, участвующих в вычислениях алгоритма, не ограничивается полиномом $p(b)$, поскольку на выходе алгоритма получается число $2^b + 1$, ограниченное показательной функцией $2^{b}$.

# Задание 7.

Выполнить компьютерное моделирование поля $\mathbb{Z}_p$ средствами ООП, перегрузив операторы \_\_add__, \_\_sub__, \_\_mul__, \_\_truediv__, \_\_pow__, \_\_neg__. Реализовать \_\_repr__, \_\_eq__. В качестве модуля $p$ генерировать общее для всех элементов поля случайное простое число длины $b$ бит. Проверить, обладает ли построенная модель указанными выше свойствами.

In [143]:
class Deduction_ring:
    m = generates_prime_number(4)  # модуль
    
    def __init__(self, number):  # для отрицательных чисел всё в порядке, так как остаток вычисляется правильно
        self.remains = number % Deduction_ring.m  # остаток от деления на модуль
    
    def __add__(self, other):
        if isinstance(other, Deduction_ring):
            if self.remains + other.remains < Deduction_ring.m:
                return self.remains + other.remains
            else:
                return (self.remains + other.remains) % Deduction_ring.m
        else:
            if self.remains + other % Deduction_ring.m < Deduction_ring.m:
                return self.remains + other % Deduction_ring.m
            else:
                return (self.remains + other.remains % Deduction_ring.m) % Deduction_ring.m
    
    def __sub__(self, other):
        if isinstance(other, Deduction_ring):
            if (self.remains + other.remains) % Deduction_ring.m == 0:
                return 0
            else:
                return 'Числа не являются обратными'
        else:
            if (self.remains + pow(other,-1,Deduction_ring.m)) % Deduction_ring.m == 0:
                return 0
            else:
                return 'Числа не являются обратными'
        
    def __mul__(self, other):
        if isinstance(other, Deduction_ring):
            if self.remains * other.remains < Deduction_ring.m:
                return self.remains * other.remains
            else:
                return (self.remains * other.remains) % Deduction_ring.m
        else:
            if self.remains * other % Deduction_ring.m < Deduction_ring.m:
                return self.remains * other % Deduction_ring.m
            else:
                return (self.remains * other % Deduction_ring.m) % Deduction_ring.m
    
    def __truediv__(self, other):
        quantity = 0  # количество "частных"
        if isinstance(other, Deduction_ring):
            for x in range(Deduction_ring.m):
                if (other.remains * x) % Deduction_ring.m == self.remains:
                    result = x
                    quantity += 1
            if quantity == 1:
                return result
            else:
                return 'Деление невозможно'
        else:
            for x in range(Deduction_ring.m):
                if (other % Deduction_ring.m * x) % Deduction_ring.m == self.remains:
                    result = x
                    quantity += 1
            if quantity == 1:
                return result
            else:
                return 'Деление невозможно'
    
    def __pow__(self, degree):
        if degree == 0:
            return 1
        elif degree == -1:
            return looking_inverse_element(self.remains, Deduction_ring.m)
        elif degree < -1:
            return looking_inverse_element((self.remains ** (-1)), abs(degree), Deduction_ring.m)
        else:
            return fast_power_leftward(self.remains, degree, Deduction_ring.m)
    
    def __neg__(self):  # поскольку в кольце не может быть отрицательных чисел, будем возвращать просто остаток
        return self.remains
    
    def __repr__(self):
        return f'{self.remains}'
    
    def __eq__(self, other):
        if isinstance(other, Deduction_ring):
            return self.remains == other.remains
        else:
            return self.remains == (other % Deduction_ring.m)

In [144]:
a = Deduction_ring(9)
b = Deduction_ring(25)
c = Deduction_ring(5)
null = Deduction_ring(0)
neutral_element = Deduction_ring(1)

Коммутативность сложения:

In [145]:
a + b == b + a

True

Ассоциативность сложения:

In [146]:
Deduction_ring(a + b) + c == a + Deduction_ring(b + c)

True

Существование нейтрального элемента относительно сложения:

In [147]:
a + null == a and null + a == a

True

Существование противоположного элемента относительно сложения:

In [148]:
b + c == null and c + b == null  # подобрать 

False

Ассоциативность умножения:

In [149]:
Deduction_ring(a * b) * c == a * Deduction_ring(b * c)

True

Дистрибутивность:

In [150]:
Deduction_ring(a * (b + c)) == Deduction_ring(a * b + a * c)

True

In [151]:
Deduction_ring(Deduction_ring(b + c) * a) == Deduction_ring(b * a + c * a)

True

Коммутативность умножения:

In [152]:
a * b == b * a

True

Существование нейтрального элемента относительно умножения:

In [153]:
neutral_element * a == a and a * neutral_element == a

True

Существование обратного элемента для ненулевых элементов:

In [154]:
b * pow(b,-1)

1